In [1]:
import requests
import dask.dataframe as dd
from datetime import datetime
import json
import pandas as pd
import os
from dotenv import load_dotenv
# from delta.tables import DeltaTable

load_dotenv()

True

In [2]:
import requests
import base64
def get_cm_access_token():
    # Credentials
    password = os.getenv("CM_API_PASSWORD", "")  # Default value for testing
    username = "cmapi"
    # Token URL
    token_url = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web/OAuth/Token"
    # Encode Basic Auth header
    credentials = f"{username}:{password}"
    encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
    # Headers and form data
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    # Token request
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        token = response.json().get("access_token")
        return token
    else:
        raise Exception(f"Failed to retrieve token: {response.status_code} - {response.text}")
# Test token retrieval
cm_access_token = get_cm_access_token()
if cm_access_token:
    print("CM token successfully retrieved")
else:
    print("Token retrieval failed")

CM token successfully retrieved


In [19]:
BASE_URL = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web"

In [25]:
headers = {
    "Authorization": f"Bearer {cm_access_token}",
    "Accept": "application/json",
}

form_id = "153770"
USER_ID = "2905"


url = f"{BASE_URL}/api/AssessmentForms/GetAssessmentFormForEdit?formId={form_id}&userId={USER_ID}"
# unfinished_form_url = f"{BASE_URL}/api/AssessmentForms/GetResidentUnfinishedForms"

In [26]:
def fetch_api(url, headers):
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    json_text = resp.text
    return json_text

In [34]:
records = []

record = fetch_api(url, headers)
if isinstance(record, dict) and "data" in record:
    records = record["data"]


# Quick sanity check:
for i, rec in enumerate(records[:5]):
    print(i, type(rec))

In [36]:
df = pd.json_normalize(
    records,
    record_path=["items", "responses"],
    meta=[
        "formId",
        "formVersion",
        "status",
        "dateCompleted",
        ["items", "templateQuestion", "text"]
    ],
    record_prefix="resp_",
    meta_prefix="meta_"
)

# 3. Rename that nested question‐text column
df = df.rename(columns={"meta_items.templateQuestion.text": "question_text"})

print(df.dtypes)


Series([], dtype: object)


In [ ]:
print(df.head(5))

   resp_questionId  resp_responseId resp_dateValue resp_stringValue  \
0              123              NaN           None             None   
1              124          30965.0           None             None   
2              125          30968.0           None             None   
3              126          30972.0           None             None   
4              127          30976.0           None             None   

   resp_numericValue resp_booleanValue resp_score  resp_recordInformation.id  \
0                NaN              True       None                    4121620   
1                NaN              None          8                    4121621   
2                NaN              None          3                    4121622   
3                NaN              None          3                    4121623   
4                NaN              None          3                    4121624   

   resp_recordInformation.userId resp_recordInformation.userNameFirst  \
0                  

In [ ]:

# then
ddf = dd.from_pandas(df, npartitions=4)  # or read JSON directly with dask.bag/json if large

In [ ]:
ddf.head(5)  # Display first 5 rows of the Dask DataFrame

,resp_questionId,resp_responseId,resp_dateValue,resp_stringValue,resp_numericValue,resp_booleanValue,resp_score,resp_recordInformation.id,resp_recordInformation.userId,resp_recordInformation.userNameFirst,resp_recordInformation.userNameLast,resp_recordInformation.userQualification,resp_recordInformation.agencyStaffName,resp_recordInformation.agencyStaffDesignation,resp_recordInformation.dateCreated,meta_formId,meta_formVersion,meta_status,meta_dateCompleted,question_text
0,123,NaN,<NA>,<NA>,NaN,True,<NA>,4121620,2821,Shuvekshya,Phuyal,Registered Nurse,<NA>,<NA>,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,PART 1 - FALLS RISK STATUS
1,124,30965.0,<NA>,<NA>,NaN,<NA>,8,4121621,2821,Shuvekshya,Phuyal,Registered Nurse,<NA>,<NA>,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk Factor: recent falls
2,125,30968.0,<NA>,<NA>,NaN,<NA>,3,4121622,2821,Shuvekshya,Phuyal,Registered Nurse,<NA>,<NA>,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: medications
3,126,30972.0,<NA>,<NA>,NaN,<NA>,3,4121623,2821,Shuvekshya,Phuyal,Registered Nurse,<NA>,<NA>,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: psychological
4,127,30976.0,<NA>,<NA>,NaN,<NA>,3,4121624,2821,Shuvekshya,Phuyal,Registered Nurse,<NA>,<NA>,2025-07-15T01:34:18.11Z,152890,1,1,2025-07-15T01:34:18Z,Risk factor: cognitive status
